[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Optimization Methods

## Discrete Optimization - Solving the KnapSack Problem

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 31/08/2025 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0012LinearFitL1.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning

# Optimization
import cvxpy as cp

# Image Processing / Computer Vision

# Miscellaneous
import math
from platform import python_version
import random

# Typing
from typing import Callable, List, Optional, Tuple, Union

# Visualization
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt

# Jupyter
from IPython import get_ipython

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

```python
valToFill = ???
```

 - Multi Line to Fill (At least one)

```python
# You need to start writing
?????
```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

# warnings.filterwarnings("ignore")

seedNum = 640 # 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
# sns.set_theme() #>! Apply SeaBorn theme
# sns.set_palette("tab10")

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2

In [ ]:
# Course Packages


In [ ]:
# Auxiliary Functions



In [ ]:
# Parameters



## The KnapSack Problem

The [KnapSack Problem](https://en.wikipedia.org/wiki/Knapsack_problem) is a well known combinatorial problem.  
It is a resource allocation problem which can be formed as an Integer (Binary) Linear Programming (ILP).

The [Knapsack Problem has many variations](https://en.wikipedia.org/wiki/List_of_knapsack_problems).  
This notebook solves the $\left\{ 0, 1 \right\}$ problem.

![](https://i.imgur.com/iargToR.png)
<!-- ![](https://i.postimg.cc/cC69GQD7/Knapsack-Problem-Illustration-svg.png) -->

> Given a set of $n$ items, each with a weight ${w}_{i}$ and a utility value ${v}_{i}$.  
> Select the sub set of items which maximizes the sum of values while not exceeding the total weight limit $W$.

* <font color='brown'>(**#**)</font> The formulation is general enough to be an effective representation of many real world use cases about optimizing choices.
* <font color='brown'>(**#**)</font> In general, the Knapsack Problem is NP Hard.
* <font color='brown'>(**#**)</font> The KnapSack problem can be effectively solved using Dynamic Programming.

## Generate Data


The data is given as:

| **Item**       | **Weight**  | **Utility** |
|----------------|-------------|-------------|
| Ant Repellent  | 1           | 2           |
| Beer           | 3           | 9           |
| Blanket        | 4           | 3           |
| Bratwurst      | 3           | 8           |
| Brownies       | 3           | 10          |
| Frisbee        | 1           | 6           |
| Salad          | 5           | 4           |
| Watermelon     | 10          | 10          |


* <font color='brown'>(**#**)</font> You may use [Kaggle - Knapsack Problem Data](https://www.kaggle.com/datasets/warcoder/knapsack-problem) for more samples and solutions.

In [ ]:
# Generate / Load the Data

lN = ['Ant Repellent', 'Beer', 'Blanket', 'Bratwurst', 'Brownies', 'Frisbee', 'Salad', 'Watermelon']
vW = np.array([1.0, 3.0, 4.0, 3.0,  3.0, 1.0, 5.0, 10.0])
vV = np.array([2.0, 9.0, 3.0, 8.0, 10.0, 6.0, 4.0, 10.0])

numItm = len(lN)

In [ ]:
# Display Data 

hF, hA = plt.subplots(figsize = (8, 8))
hA.bar(np.arange(numItm), vW, width = 0.25, align = 'edge', color = 'b', alpha = 0.75, label = 'Weight')
hA.bar(np.arange(numItm), vV, width = -0.25, align = 'edge', color = 'r', alpha = 0.75, label = 'Utility')
hA.set_xticks(np.arange(numItm))
hA.set_xticklabels(lN, rotation = 45, ha = 'right')
hA.set_xlabel('Item Index')
hA.legend();

* <font color='red'>(**?**)</font> Think of a model to solve the problem with _Integer Programming_.

## Solve the Linear Programming Problem

In [ ]:
# DCP Solution

#===========================Fill This===========================#
# 1. Formulate the problem in CVXPY.  
#    Use `vX` for the optimal argument.
# !! Pay attention to enforce a valid answer.

# Model Data
vX = cp.Variable(numItm) #<! Variable

# Model Problem
cpObjFun = cp.Maximize(vV @ vX) #<! Objective Function
cpConst  = [vX >= 0, vX <= 0] #<! Constraints
oCvxPrb  = cp.Problem(cpObjFun, cpConst) #<! Problem

oCvxPrb.solve(solver = cp.SCIP)
#===============================================================#

vXLP    = vX.value
vXLPRnd = np.round(vX.value)

assert (oCvxPrb.status == 'optimal'), 'The problem is not solved.'
print('Problem is solved.')

## Solve the Integer Linear Programming

In [ ]:
# DCP Solution

#===========================Fill This===========================#
# 1. Formulate the problem in CVXPY.  
#    Use `vX` for the optimal argument.
# !! You may find `cp.Variable(<>, boolean = True)` useful.

# Model Data
vX = cp.Variable(numItm, boolean = True) #<! Variable

# Model Problem
cpObjFun = cp.Maximize(vV @ vX) #<! Objective Function
cpConst  = [vX >= 0, vX <= 0] #<! Constraints
oCvxPrb  = cp.Problem(cpObjFun, cpConst) #<! Problem

oCvxPrb.solve(solver = cp.SCIP)
#===============================================================#

vXILP = vX.value

assert (oCvxPrb.status == 'optimal'), 'The problem is not solved.'
print('Problem is solved.')

In [ ]:
# Plot Solutions

hFig, hAx = plt.subplots(figsize = (8, 8))
hAx.bar(np.arange(numItm), vW, width = 0.25, align = 'edge', color = 'b', alpha = 0.75, label = 'Weight')
hAx.bar(np.arange(numItm), vV, width = -0.25, align = 'edge', color = 'r', alpha = 0.75, label = 'Utility')
hAx.set_xticks(np.arange(numItm))

* <font color='red'>(**?**)</font> Explain the solution. Is it guaranteed to hold all constraints? Explain.